In [1]:
%pwd

'c:\\projects\\online_fraud_detection_app\\research'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'c:\\projects\\online_fraud_detection_app'

In [4]:
import pandas as pd 
df=pd.read_csv("C:\\online-fraud-detection\\dataset\\onlinefraud.csv")


In [5]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [7]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [8]:
df.drop_duplicates(inplace=True)
print(df.duplicated().any())

False


In [9]:
df=df.drop(columns=['nameOrig','nameDest','isFlaggedFraud','step'],axis=1)

In [10]:
df.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,9839.64,170136.00,160296.36,0.0,0.00,0
1,PAYMENT,1864.28,21249.00,19384.72,0.0,0.00,0
2,TRANSFER,181.00,181.00,0.00,0.0,0.00,1
3,CASH_OUT,181.00,181.00,0.00,21182.0,0.00,1
4,PAYMENT,11668.14,41554.00,29885.86,0.0,0.00,0
5,PAYMENT,7817.71,53860.00,46042.29,0.0,0.00,0
6,PAYMENT,7107.77,183195.00,176087.23,0.0,0.00,0
7,PAYMENT,7861.64,176087.23,168225.59,0.0,0.00,0
8,PAYMENT,4024.36,2671.00,0.00,0.0,0.00,0
9,DEBIT,5337.77,41720.00,36382.23,41898.0,40348.79,0


In [11]:
df["type"].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [12]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
df["type"]=label.fit_transform(df['type'])



In [13]:
df.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,3,9839.64,170136.00,160296.36,0.0,0.00,0
1,3,1864.28,21249.00,19384.72,0.0,0.00,0
2,4,181.00,181.00,0.00,0.0,0.00,1
3,1,181.00,181.00,0.00,21182.0,0.00,1
4,3,11668.14,41554.00,29885.86,0.0,0.00,0
5,3,7817.71,53860.00,46042.29,0.0,0.00,0
6,3,7107.77,183195.00,176087.23,0.0,0.00,0
7,3,7861.64,176087.23,168225.59,0.0,0.00,0
8,3,4024.36,2671.00,0.00,0.0,0.00,0
9,2,5337.77,41720.00,36382.23,41898.0,40348.79,0


In [14]:
correlation_matrix = df.corr()
sorted_correlations = correlation_matrix['isFraud'].sort_values(ascending=False)
sorted_correlations

isFraud           1.000000
amount            0.076688
type              0.020833
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64

In [15]:
from sklearn.model_selection import train_test_split
import numpy as np
df['isFraud']=df['isFraud'].map({0:"no fraud",1:"isfraud"})
x=np.array(df[['type','amount','oldbalanceOrg','newbalanceOrig']])
y=df['isFraud']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
from joblib import dump
from sklearn.ensemble import RandomForestClassifier

# Create and train your model
model = RandomForestClassifier(n_estimators=10)
model.fit(x_train, y_train)

model_score=model.score(x_test,y_test)
print(f"Model Score: {model_score}")
dump(model, 'random_forest_model.joblib') 


Model Score: 0.9997170976735998


['random_forest_model.joblib']

In [17]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(x_test)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

     isfraud       0.88      0.90      0.89      1620
    no fraud       1.00      1.00      1.00   1270904

    accuracy                           1.00   1272524
   macro avg       0.94      0.95      0.94   1272524
weighted avg       1.00      1.00      1.00   1272524

